# Saving for retirement with inflation
- Let's assume that we want to consume a constant **real** amount in retirement
  - our standard of living will be unchanged each year
- We can either finance retirement in nominal terms or in real terms

We need a few relationships to work on this.  We'll assume a constant inflation rate and that the base year is time 0.

- First, what is the relationship between real and nominal cash flows at a point in time $t$?
$$ \text{Nominal CF}_t = \text{Real CF}_t \cdot (1+\text{inflation})^t. $$

- Second, what is the relationship between real and nominal rates of return?
$$ 1+ \text{nominal rate} = (1+\text{real rate})(1+\text{inflation}).$$

In [1]:
import numpy as np
import pandas as pd
import numpy_financial as npf
import plotly.graph_objects as go
pd.options.display.float_format = '${:,.2f}'.format

In [2]:
# We'll set up 2 bank accounts (1 in nominal terms and 1 in real terms)
NOMINAL    = 0.05
INFLATION  = 0.02
N_SAVING   = 30
N_WITHDRAW = 20
PV  = 0.0
WITHDRAWAL = 100000

column_index = pd.MultiIndex.from_product([['Nominal','Real'],['begbal','capgain','dep/with','endbal']])
idx = np.arange(1,N_SAVING + N_WITHDRAW + 1)

acct = pd.DataFrame(dtype=float,columns=column_index,index=idx)
acct

Nominal                           Real                        
    begbal capgain dep/with endbal begbal capgain dep/with endbal
1      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
2      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
3      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
4      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
5      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
6      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
7      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
8      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
9      NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
10     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
11     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
12     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
13     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
14     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
15     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
16     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
17     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
18     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
19     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
20     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
21     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
22     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
23     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
24     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
25     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
26     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
27     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
28     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
29     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
30     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
31     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
32     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
33     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
34     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
35     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
36     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
37     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
38     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
39     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
40     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
41     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
42     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
43     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
44     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
45     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
46     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
47     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
48     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
49     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN
50     NaN     NaN      NaN    NaN    NaN     NaN      NaN    NaN

In [3]:
# Let's start by saving a constant real amount each year.
# This means that we can solve the problem entirely in real terms.
real = (1+NOMINAL)/(1+INFLATION)-1
print(f'{real:,.2%}')

2.94%


In [4]:
# First, we find how much we'd need to have saved as of t=N_SAVING:
end_savings_bal = npf.pv(real,N_WITHDRAW,-WITHDRAWAL,0)
print(f'{end_savings_bal:,.2f}')

1,495,870.98


In [5]:
# Second, we solve a PMT problem to determine the amount we need to save:

pmt = npf.pmt(real,N_SAVING,PV,-end_savings_bal)
print(f'{pmt:,.2f}')

31,742.87


In [6]:
# Fill out the real side of the balance sheet
acct[('Real','dep/with')] = np.where(acct.index <= N_SAVING, pmt,-WITHDRAWAL)
for t in acct.index:
    if t==1:
        acct.loc[t,('Real','begbal')] = PV
    else:
        acct.loc[t,('Real','begbal')] = acct.loc[t-1,('Real','endbal')]
    acct.loc[t,('Real','capgain')] = acct.loc[t,('Real','begbal')]*real
    acct.loc[t,('Real','endbal')]  = acct.loc[t,('Real','begbal')] + acct.loc[t,('Real','capgain')] + acct.loc[t,('Real','dep/with')] 
acct

Nominal                                  Real                          \
    begbal capgain dep/with endbal        begbal    capgain     dep/with   
1      NaN     NaN      NaN    NaN         $0.00      $0.00   $31,742.87   
2      NaN     NaN      NaN    NaN    $31,742.87    $933.61   $31,742.87   
3      NaN     NaN      NaN    NaN    $64,419.35  $1,894.69   $31,742.87   
4      NaN     NaN      NaN    NaN    $98,056.91  $2,884.03   $31,742.87   
5      NaN     NaN      NaN    NaN   $132,683.80  $3,902.46   $31,742.87   
6      NaN     NaN      NaN    NaN   $168,329.14  $4,950.86   $31,742.87   
7      NaN     NaN      NaN    NaN   $205,022.86  $6,030.08   $31,742.87   
8      NaN     NaN      NaN    NaN   $242,795.81  $7,141.05   $31,742.87   
9      NaN     NaN      NaN    NaN   $281,679.74  $8,284.70   $31,742.87   
10     NaN     NaN      NaN    NaN   $321,707.30  $9,461.98   $31,742.87   
11     NaN     NaN      NaN    NaN   $362,912.15 $10,673.89   $31,742.87   
12     NaN     NaN      NaN    NaN   $405,328.91 $11,921.44   $31,742.87   
13     NaN     NaN      NaN    NaN   $448,993.21 $13,205.68   $31,742.87   
14     NaN     NaN      NaN    NaN   $493,941.76 $14,527.70   $31,742.87   
15     NaN     NaN      NaN    NaN   $540,212.33 $15,888.60   $31,742.87   
16     NaN     NaN      NaN    NaN   $587,843.80 $17,289.52   $31,742.87   
17     NaN     NaN      NaN    NaN   $636,876.19 $18,731.65   $31,742.87   
18     NaN     NaN      NaN    NaN   $687,350.71 $20,216.20   $31,742.87   
19     NaN     NaN      NaN    NaN   $739,309.78 $21,744.41   $31,742.87   
20     NaN     NaN      NaN    NaN   $792,797.05 $23,317.56   $31,742.87   
21     NaN     NaN      NaN    NaN   $847,857.48 $24,936.98   $31,742.87   
22     NaN     NaN      NaN    NaN   $904,537.34 $26,604.04   $31,742.87   
23     NaN     NaN      NaN    NaN   $962,884.24 $28,320.12   $31,742.87   
24     NaN     NaN      NaN    NaN $1,022,947.24 $30,086.68   $31,742.87   
25     NaN     NaN      NaN    NaN $1,084,776.79 $31,905.20   $31,742.87   
26     NaN     NaN      NaN    NaN $1,148,424.86 $33,777.20   $31,742.87   
27     NaN     NaN      NaN    NaN $1,213,944.93 $35,704.26   $31,742.87   
28     NaN     NaN      NaN    NaN $1,281,392.06 $37,688.00   $31,742.87   
29     NaN     NaN      NaN    NaN $1,350,822.93 $39,730.09   $31,742.87   
30     NaN     NaN      NaN    NaN $1,422,295.88 $41,832.23   $31,742.87   
31     NaN     NaN      NaN    NaN $1,495,870.98 $43,996.21 $-100,000.00   
32     NaN     NaN      NaN    NaN $1,439,867.19 $42,349.04 $-100,000.00   
33     NaN     NaN      NaN    NaN $1,382,216.23 $40,653.42 $-100,000.00   
34     NaN     NaN      NaN    NaN $1,322,869.64 $38,907.93 $-100,000.00   
35     NaN     NaN      NaN    NaN $1,261,777.57 $37,111.11 $-100,000.00   
36     NaN     NaN      NaN    NaN $1,198,888.68 $35,261.43 $-100,000.00   
37     NaN     NaN      NaN    NaN $1,134,150.11 $33,357.36 $-100,000.00   
38     NaN     NaN      NaN    NaN $1,067,507.47 $31,397.28 $-100,000.00   
39     NaN     NaN      NaN    NaN   $998,904.75 $29,379.55 $-100,000.00   
40     NaN     NaN      NaN    NaN   $928,284.30 $27,302.48 $-100,000.00   
41     NaN     NaN      NaN    NaN   $855,586.78 $25,164.32 $-100,000.00   
42     NaN     NaN      NaN    NaN   $780,751.09 $22,963.27 $-100,000.00   
43     NaN     NaN      NaN    NaN   $703,714.36 $20,697.48 $-100,000.00   
44     NaN     NaN      NaN    NaN   $624,411.84 $18,365.05 $-100,000.00   
45     NaN     NaN      NaN    NaN   $542,776.90 $15,964.03 $-100,000.00   
46     NaN     NaN      NaN    NaN   $458,740.92 $13,492.38 $-100,000.00   
47     NaN     NaN      NaN    NaN   $372,233.30 $10,948.04 $-100,000.00   
48     NaN     NaN      NaN    NaN   $283,181.34  $8,328.86 $-100,000.00   
49     NaN     NaN      NaN    NaN   $191,510.20  $5,632.65 $-100,000.00   
50     NaN     NaN      NaN    NaN    $97,142.86  $2,857.14 $-100,000.00   

                  
          endbal  
1     $31,74

In [7]:
# Fill our the nominal side of the balance sheet
acct[('Nominal','dep/with')] = acct[('Real','dep/with')] *(1+INFLATION)**acct.index
for t in acct.index:
    if t==1:
        acct.loc[t,('Nominal','begbal')] = PV
    else:
        acct.loc[t,('Nominal','begbal')] = acct.loc[t-1,('Nominal','endbal')]
    acct.loc[t,('Nominal','capgain')] = acct.loc[t,('Nominal','begbal')]*NOMINAL
    acct.loc[t,('Nominal','endbal')]  = acct.loc[t,('Nominal','begbal')] + acct.loc[t,('Nominal','capgain')] + acct.loc[t,('Nominal','dep/with')] 
acct

Nominal                                                 Real  \
          begbal     capgain     dep/with        endbal        begbal   
1          $0.00       $0.00   $32,377.73    $32,377.73         $0.00   
2     $32,377.73   $1,618.89   $33,025.28    $67,021.89    $31,742.87   
3     $67,021.89   $3,351.09   $33,685.79   $104,058.77    $64,419.35   
4    $104,058.77   $5,202.94   $34,359.50   $143,621.21    $98,056.91   
5    $143,621.21   $7,181.06   $35,046.69   $185,848.97   $132,683.80   
6    $185,848.97   $9,292.45   $35,747.63   $230,889.04   $168,329.14   
7    $230,889.04  $11,544.45   $36,462.58   $278,896.07   $205,022.86   
8    $278,896.07  $13,944.80   $37,191.83   $330,032.71   $242,795.81   
9    $330,032.71  $16,501.64   $37,935.67   $384,470.01   $281,679.74   
10   $384,470.01  $19,223.50   $38,694.38   $442,387.89   $321,707.30   
11   $442,387.89  $22,119.39   $39,468.27   $503,975.55   $362,912.15   
12   $503,975.55  $25,198.78   $40,257.63   $569,431.96   $405,328.91   
13   $569,431.96  $28,471.60   $41,062.79   $638,966.34   $448,993.21   
14   $638,966.34  $31,948.32   $41,884.04   $712,798.70   $493,941.76   
15   $712,798.70  $35,639.94   $42,721.72   $791,160.36   $540,212.33   
16   $791,160.36  $39,558.02   $43,576.16   $874,294.53   $587,843.80   
17   $874,294.53  $43,714.73   $44,447.68   $962,456.94   $636,876.19   
18   $962,456.94  $48,122.85   $45,336.63 $1,055,916.42   $687,350.71   
19 $1,055,916.42  $52,795.82   $46,243.37 $1,154,955.60   $739,309.78   
20 $1,154,955.60  $57,747.78   $47,168.23 $1,259,871.62   $792,797.05   
21 $1,259,871.62  $62,993.58   $48,111.60 $1,370,976.80   $847,857.48   
22 $1,370,976.80  $68,548.84   $49,073.83 $1,488,599.47   $904,537.34   
23 $1,488,599.47  $74,429.97   $50,055.31 $1,613,084.75   $962,884.24   
24 $1,613,084.75  $80,654.24   $51,056.41 $1,744,795.39 $1,022,947.24   
25 $1,744,795.39  $87,239.77   $52,077.54 $1,884,112.71 $1,084,776.79   
26 $1,884,112.71  $94,205.64   $53,119.09 $2,031,437.43 $1,148,424.86   
27 $2,031,437.43 $101,571.87   $54,181.47 $2,187,190.78 $1,213,944.93   
28 $2,187,190.78 $109,359.54   $55,265.10 $2,351,815.42 $1,281,392.06   
29 $2,351,815.42 $117,590.77   $56,370.40 $2,525,776.59 $1,350,822.93   
30 $2,525,776.59 $126,288.83   $57,497.81 $2,709,563.24 $1,422,295.88   
31 $2,709,563.24 $135,478.16 $-184,758.88 $2,660,282.52 $1,495,870.98   
32 $2,660,282.52 $133,014.13 $-188,454.06 $2,604,842.58 $1,439,867.19   
33 $2,604,842.58 $130,242.13 $-192,223.14 $2,542,861.57 $1,382,216.23   
34 $2,542,861.57 $127,143.08 $-196,067.60 $2,473,937.05 $1,322,869.64   
35 $2,473,937.05 $123,696.85 $-199,988.96 $2,397,644.94 $1,261,777.57   
36 $2,397,644.94 $119,882.25 $-203,988.73 $2,313,538.46 $1,198,888.68   
37 $2,313,538.46 $115,676.92 $-208,068.51 $2,221,146.87 $1,134,150.11   
38 $2,221,146.87 $111,057.34 $-212,229.88 $2,119,974.34 $1,067,507.47   
39 $2,119,974.34 $105,998.72 $-216,474.48 $2,009,498.58   $998,904.75   
40 $2,009,498.58 $100,474.93 $-220,803.97 $1,889,169.54   $928,284.30   
41 $1,889,169.54  $94,458.48 $-225,220.05 $1,758,407.97   $855,586.78   
42 $1,758,407.97  $87,920.40 $-229,724.45 $1,616,603.92   $780,751.09   
43 $1,616,603.92  $80,830.20 $-234,318.94 $1,463,115.18   $703,714.36   
44 $1,463,115.18  $73,155.76 $-239,005.31 $1,297,265.63   $624,411.84   
45 $1,297,265.63  $64,863.28 $-243,785.42 $1,118,343.49   $542,776.90   
46 $1,118,343.49  $55,917.17 $-248,661.13   $925,599.53   $458,740.92   
47   $925,599.53  $46,279.98 $-253,634.35   $718,245.16   $372,233.30   
48   $718,245.16  $35,912.26 $-258,707.04   $495,450.38   $283,181.34   
49   $495,450.38  $24,772.52 $-263,881.18   $256,341.72   $191,510.20   
50   $256,341.72  $12,817.09 $-269,158.80        $-0.00    $97,142.86   

                                          
      capgain     dep/with        endbal  
1       $0.00   $31,742.87    $31,742.87  
2     $933.61   $31,742.87    $64,419.35  
3   $1,894.69   $31,742.87    $98,056.91

### Excercise: what if we had wanted to save in constant nominal terms?

## Uncertain returns

In [8]:
from scipy.stats import norm
norm.rvs(loc=0.04, scale = 0.1, size=30, random_state=10)

array([ 0.17315865,  0.1115279 , -0.11454003,  0.03916162,  0.1021336 ,
       -0.03200856,  0.06655116,  0.05085485,  0.04042914,  0.02253998,
        0.08330262,  0.16030374, -0.05650657,  0.14282741,  0.06286301,
        0.08451376, -0.07366022,  0.05351369,  0.1884537 , -0.06798049,
       -0.15777283, -0.13433723,  0.06660702,  0.27849673,  0.15236913,
        0.20726222,  0.04991492,  0.17979964,  0.0128752 ,  0.10132042])

In [9]:
MEAN       = 0.03
SD         = 0.10
N_SAVING   = 30
N_WITHDRAW = 20
PMT        = 32000.0
WITHDRAWAL = 100000.0
PV         = 0.0

acct = pd.DataFrame(dtype=float,columns=['begbal','return','capgain','dep/with','endbal'],index=np.arange(1,N_SAVING + N_WITHDRAW + 1))
acct['return'] = norm.rvs(loc=MEAN, scale = SD, size=N_SAVING+N_WITHDRAW, random_state=10)

acct['dep/with'] = np.where(acct.index <= N_SAVING, PMT,-WITHDRAWAL)
for t in acct.index:
    if t==1:
        acct.loc[t,'begbal'] = PV
    else:
        acct.loc[t,'begbal'] = acct.loc[t-1,'endbal']
    acct.loc[t,'capgain'] = acct.loc[t,'begbal']*acct.loc[t,'return']
    acct.loc[t,'endbal']  = acct.loc[t,'begbal'] + acct.loc[t,'capgain'] + acct.loc[t,'dep/with'] 
acct.loc[N_SAVING + N_WITHDRAW,'endbal']

911801.0357697688

But this was just a single possible realization.  We want to know what the **distribution** of possible balances look like.

Let's run the simulation above 1000 times and collect the ending balance for each month.

In [10]:
N_SIMS=1000
sims = pd.DataFrame(dtype=float,columns=['endbal'],index=np.arange(N_SIMS))
def simulate(MEAN,SD):
    acct = pd.DataFrame(dtype=float,columns=['begbal','return','capgain','dep/with','endbal'],index=np.arange(1,N_SAVING + N_WITHDRAW + 1))
    acct['return'] = norm.rvs(loc=MEAN, scale = SD, size=N_SAVING+N_WITHDRAW)

    acct['dep/with'] = np.where(acct.index <= N_SAVING, PMT,-WITHDRAWAL)
    for t in acct.index:
        if t==1:
            acct.loc[t,'begbal'] = PV
        else:
            acct.loc[t,'begbal'] = acct.loc[t-1,'endbal']
        acct.loc[t,'capgain'] = acct.loc[t,'begbal']*acct.loc[t,'return']
        acct.loc[t,'endbal']  = acct.loc[t,'begbal'] + acct.loc[t,'capgain'] + acct.loc[t,'dep/with'] 
    return acct.loc[N_SAVING + N_WITHDRAW,'endbal']
for s in sims.index:
    sims.loc[s] = simulate(MEAN,SD)

In [11]:
# How many times did we run out of money?
frac_pos = (sims.endbal >0).mean()
print(frac_pos)

0.411


In [12]:
# Plot the distribution of ending balances
fig = go.Figure()
trace= go.Histogram(x=sims.endbal, histnorm='percent',hovertemplate="<br>%{y:.2}% of simulations <br><extra></extra>")
fig.add_trace(trace)
# some formatting
fig.update_traces(marker_line_width=1, marker_line_color='black')
fig.layout.xaxis["title"] = "Ending Balance"
fig.layout.yaxis["title"] = "Percent of Simulations"
fig.add_vline(x=0, line_width=4, line_dash="dash", line_color="black")
fig.add_annotation(x=sims.endbal.max()*0.7, y=8,
            text="Ending balance positive in <br>"+f'{frac_pos:.1%} of simulations', showarrow=False)
fig.show()